In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import openai


In [ ]:
openai.api_key=os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)


In [ ]:
from langchain_community.utilities import SQLDatabase

In [ ]:
db_uri = (
    f"postgresql+psycopg2://{os.getenv('POSTGRES_USER')}:"
    f"{os.getenv('POSTGRES_PASSWORD')}@localhost:5433/"
    f"{os.getenv('POSTGRES_DB')}"
)

In [ ]:
db = SQLDatabase.from_uri(db_uri, include_tables=None)

In [ ]:
db.run("SELECT *  FROM person")

In [ ]:
def get_schema(_):
    return db.get_table_info()

In [ ]:
get_schema(None)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)


In [ ]:
user_question = 'how many albums are there in the database?'
sql_chain.invoke({"question": user_question})